{'Midwest', 'Northeast', 'Southeast', 'Southwest', 'West'}

In [222]:
# CQL to create states as nodes
# CQL to query the difference in population from a given state to all the other states.
# Create relationships between the states as defined above (i.e. region and the binary classifications).
# Use the above states_df to read the data (i.e. please don't repeatedly write out state and/or field names).


# import the neo4j driver for Python
from neo4j import GraphDatabase
from scipy import stats

# Database Credentials
uri = "bolt://localhost:7687"
userName = "neo4j"
password = "mentor-union-parody-marco-battery-5682"

# Connect to the neo4j database server
graphDB_Driver = GraphDatabase.driver(uri, auth=(userName, password))

regions = {'Maine': 'Northeast', 'Massachusetts': 'Northeast', 'Rhode Island': 'Northeast', 'Connecticut': 'Northeast', 'New Hampshire': 'Northeast', 'Vermont': 'Northeast', 'New York': 'Northeast', 'Pennsylvania': 'Northeast', 'New Jersey': 'Northeast', 'Delaware': 'Northeast', 'Maryland': 'Northeast', 'West Virginia': 'Southeast', 'Virginia': 'Southeast', 'Kentucky': 'Southeast', 'Tennessee': 'Southeast', 'North Carolina': 'Southeast', 'South Carolina': 'Southeast', 'Georgia': 'Southeast', 'Alabama': 'Southeast', 'Mississippi': 'Southeast', 'Arkansas': 'Southeast', 'Louisiana': 'Southeast', 'Florida': 'Southeast', 'Ohio': 'Midwest', 'Indiana': 'Midwest', 'Michigan': 'Midwest', 'Illinois': 'Midwest', 'Missouri': 'Midwest', 'Wisconsin': 'Midwest', 'Minnesota': 'Midwest', 'Iowa': 'Midwest', 'Kansas': 'Midwest', 'Nebraska': 'Midwest', 'South Dakota': 'Midwest', 'North Dakota': 'Midwest', 'Texas': 'Southwest', 'Oklahoma': 'Southwest', 'New Mexico': 'Southwest', 'Arizona': 'Southwest', 'Colorado': 'West', 'Wyoming': 'West', 'Montana': 'West', 'Idaho': 'West', 'Washington': 'West', 'Oregon': 'West', 'Utah': 'West', 'Nevada': 'West', 'California': 'West', 'Alaska': 'West', 'Hawaii': 'West'}

# Function to create nodes
def create_nodes(states_df):
    queries = []
    #delete everything first
    cqlCreate = "MATCH (n) DETACH DELETE n"
    with graphDB_Driver.session() as graphDB_Session:
            graphDB_Session.run(cqlCreate)
    # Create states as nodes
    for i, row in states_df.iterrows():
        state = row['States']
        poverty = row['Poverty']
        infant_mort = row['Infant Mort']
        white = row['White']
        crime = row['Crime']
        doctors = row['Doctors']
        traf_deaths = row['Traf Deaths']
        university = row['University']
        unemployed = row['Unemployed']
        income = row['Income']
        population = row['Population']
        region = row['region']
                
        poverty_bin = row['poverty_bin']
        infant_mort_bin = row['infant_mort_bin']
        white_bin = row['white_bin']
        crime_bin = row['crime_bin']
        doctors_bin = row['doctors_bin']
        traf_deaths_bin = row['traf_deaths_bin']
        university_bin = row['university_bin']
        unemployed_bin = row['unemployed_bin']
        income_bin = row['income_bin']
        population_bin = row['population_bin']
        
        # Assign region to each state
        
        # Create CQL query to create each state node
        cqlCreate = "CREATE (" + state.replace(" ", "") + ":state { name: '" + state + f"', poverty: {poverty}" + f", infant_mort: {infant_mort}" + f", white: {white}" + f", crime: {crime}" +\
        f", doctors: {doctors}" + f", traf_deaths: {traf_deaths}"  + f", university: {university}" + f", unemployed: {unemployed}" + f", income: {income}" + f", population: {population}" + f", region: '{region}'" +\
        f", poverty_bin: {poverty_bin}" + f", infant_mort_bin: {infant_mort_bin}" + f", white_bin: {white_bin}" + f", crime_bin: {crime_bin}" + f", doctors_bin: {doctors_bin}" + f", traf_deaths_bin: {traf_deaths_bin}" + \
        f", university_bin: {university_bin}" + f", unemployed_bin: {unemployed_bin}" + f", income_bin: {income_bin}" + f", population_bin: {population_bin}" + "})"
        queries.append(cqlCreate)
                
        # Execute the CQL query
        with graphDB_Driver.session() as graphDB_Session:
            graphDB_Session.run(cqlCreate)
            
    # Create region nodes
    #{'Midwest', 'Northeast', 'Southeast', 'Southwest', 'West'}
    cqlCreate = "CREATE (Midwest:region {name: 'Midwest'}), (Northeast:region {name: 'Northeast'}), (Southeast:region {name: 'Southeast'}), (Southwest:region {name: 'Southwest'}), (West:region {name: 'West'})"
    with graphDB_Driver.session() as graphDB_Session:
        graphDB_Session.run(cqlCreate)
        
    # Create nodes for factors
    cqlCreate = "CREATE (poverty:factor {name: 'Poverty'}), (infant_mort:factor {name: 'Infant Mort'}), (white:factor {name: 'White'}), (crime:factor {name: 'Crime'}), (doctors:factor {name: 'Doctors'}), (traf_deaths:factor {name: 'Traf Deaths'}), (university:factor {name: 'University'}), (unemployed:factor {name: 'Unemployed'}), (income:factor {name: 'Income'}), (population:factor {name: 'Population'})"
    with graphDB_Driver.session() as graphDB_Session:
        graphDB_Session.run(cqlCreate)
                
# Function to create relationships between the states
def create_relationships(states_df):
    cqlCreate = ""
    # Create state and region relationships
    for i, row in states_df.iterrows():
        state = row['States']
        region = row['region']
        cqlCreate = f"MATCH (x:state {{name: '{state}'}}), (y:region {{name: '{region}'}}) CREATE (x)-[:part_of]->(y)"
        with graphDB_Driver.session() as graphDB_Session:
            graphDB_Session.run(cqlCreate)
    
    # Create state and factor relationships
    for i, row in states_df.iterrows():
        state = row['States']
        poverty_bin = row['poverty_bin']
        infant_mort_bin = row['infant_mort_bin']
        white_bin = row['white_bin']
        crime_bin = row['crime_bin']
        doctors_bin = row['doctors_bin']
        traf_deaths_bin = row['traf_deaths_bin']
        university_bin = row['university_bin']
        unemployed_bin = row['unemployed_bin']
        income_bin = row['income_bin']
        population_bin = row['population_bin']
        
        # Create CQL query to create relationships
        if poverty_bin == 1:
            cqlCreate = f"MATCH (x:state {{name: '{state}'}}), (y:factor {{name: 'Poverty'}}) CREATE (x)-[:has]->(y)"
            with graphDB_Driver.session() as graphDB_Session:
                graphDB_Session.run(cqlCreate)
        if infant_mort_bin == 1:
            cqlCreate = f"MATCH (x:state {{name: '{state}'}}), (y:factor {{name: 'Infant Mort'}}) CREATE (x)-[:has]->(y)"
            with graphDB_Driver.session() as graphDB_Session:
                graphDB_Session.run(cqlCreate)            
        if white_bin == 1:
            cqlCreate = f"MATCH (x:state {{name: '{state}'}}), (y:factor {{name: 'White'}}) CREATE (x)-[:has]->(y)"
            with graphDB_Driver.session() as graphDB_Session:
                graphDB_Session.run(cqlCreate)
        if crime_bin == 1:
            cqlCreate = f"MATCH (x:state {{name: '{state}'}}), (y:factor {{name: 'Crime'}}) CREATE (x)-[:has]->(y)"
            with graphDB_Driver.session() as graphDB_Session:
                graphDB_Session.run(cqlCreate)
        if doctors_bin == 1:
            cqlCreate = f"MATCH (x:state {{name: '{state}'}}), (y:factor {{name: 'Doctors'}}) CREATE (x)-[:has]->(y)"
            with graphDB_Driver.session() as graphDB_Session:
                graphDB_Session.run(cqlCreate)
        if traf_deaths_bin == 1:
            cqlCreate = f"MATCH (x:state {{name: '{state}'}}), (y:factor {{name: 'Traf Deaths'}}) CREATE (x)-[:has]->(y)"
            with graphDB_Driver.session() as graphDB_Session:
                graphDB_Session.run(cqlCreate)
        if university_bin == 1:
            cqlCreate = f"MATCH (x:state {{name: '{state}'}}), (y:factor {{name: 'University'}}) CREATE (x)-[:has]->(y)"
            with graphDB_Driver.session() as graphDB_Session:
                graphDB_Session.run(cqlCreate)
        if unemployed_bin == 1:
            cqlCreate = f"MATCH (x:state {{name: '{state}'}}), (y:factor {{name: 'Unemployed'}}) CREATE (x)-[:has]->(y)"
            with graphDB_Driver.session() as graphDB_Session:
                graphDB_Session.run(cqlCreate)
        if income_bin == 1:
            cqlCreate = f"MATCH (x:state {{name: '{state}'}}), (y:factor {{name: 'Income'}}) CREATE (x)-[:has]->(y)"
            with graphDB_Driver.session() as graphDB_Session:
                graphDB_Session.run(cqlCreate)
        if population_bin == 1:
            cqlCreate = f"MATCH (x:state {{name: '{state}'}}), (y:factor {{name: 'Population'}}) CREATE (x)-[:has]->(y)"
            with graphDB_Driver.session() as graphDB_Session:
                graphDB_Session.run(cqlCreate)
            
        # Execute the CQL query
        #with graphDB_Driver.session() as graphDB_Session:
            #graphDB_Session.run(cqlCreate)


# Main
if __name__ == '__main__':
    # Read the data
    states_df = pd.read_csv('https://raw.githubusercontent.com/thistleknot/python-ml/master/data/raw/states.csv')
    
    mads = stats.median_abs_deviation(states_df.iloc[:,1:11])
    mads_df = pd.DataFrame([mads],columns=states_df.columns[1:11])
    
    states_df['poverty_bin'] = np.where(states_df['Poverty'] <= (states_df['Poverty'].median()-mads_df['Poverty'].values[0]), 1, 0)
    states_df['infant_mort_bin'] = np.where(states_df['Infant Mort'] <= (states_df['Infant Mort'].median()-mads_df['Infant Mort'].values[0]), 1, 0)
    states_df['white_bin'] = np.where(states_df['White'] >= (states_df['White'].median()+mads_df['White'].values[0]), 1, 0)
    states_df['crime_bin'] = np.where(states_df['Crime'] <= (states_df['Crime'].median()-mads_df['Crime'].values[0]), 1, 0)
    states_df['doctors_bin'] = np.where(states_df['Doctors'] >= (states_df['Doctors'].median()+mads_df['Doctors'].values[0]), 1, 0)
    states_df['traf_deaths_bin'] = np.where(states_df['Traf Deaths'] <= (states_df['Traf Deaths'].median()-mads_df['Traf Deaths'].values[0]), 1, 0)
    states_df['university_bin'] = np.where(states_df['University'] >= (states_df['University'].median()+mads_df['University'].values[0]), 1, 0)
    states_df['unemployed_bin'] = np.where(states_df['Unemployed'] <= (states_df['Unemployed'].median()-mads_df['Unemployed'].values[0]), 1, 0)
    states_df['income_bin'] = np.where(states_df['Income'] >= (states_df['Income'].median()+mads_df['Income'].values[0]), 1, 0)
    states_df['population_bin'] = np.where(states_df['Population'] <= (states_df['Population'].median()-mads_df['Population'].values[0]), 1, 0)
    states_df['region'] = [regions[s] for s in states_df['States'].values]
    
    # Create states as nodes
    create_nodes(states_df)
    # Query the difference in population from a given state to all the other states
    create_relationships(states_df)
    #query_state_pop_diff('California')
    # Create relationships between the states
    

2.25